# 1. Import libraries

In [1]:
import requests
from pandas import json_normalize
import numpy as np 
import pandas as pd

# Importing the cleaning functions:


# 2. Import data

In [2]:
smart = pd.read_csv("../data/clean_smart.csv",encoding = "ISO-8859-1")

In [3]:
smart

,Unnamed: 0,City,Country,Smart_Mobility,Smart_Environment,Smart_Government,Smart_Economy,Smart_People,Smart_Living,SmartCity_Index
0,0,Oslo,Norway,6480,6512,7516,4565,8618,9090,7138
1,1,Bergen,Norway,7097,6876,7350,4905,8050,9090,7296
2,2,Amsterdam,Netherlands,7540,5558,8528,8095,7098,7280,7311
3,3,Copenhagen,Denmark,7490,7920,8726,5580,5780,7200,7171
4,4,Stockholm,Sweden,6122,7692,8354,4330,6743,7730,6812
...,...,...,...,...,...,...,...,...,...,...
97,97,Riga,Latvia,4152,4584,4616,7380,3745,4330,4712
98,98,Beijing,China,7610,2998,2806,4905,5183,1980,4449
99,99,St Petersburg,Russia,4588,2908,3622,4515,5390,4100,4191
100,100,Calgary,Canada,6675,4052,5946,8022,6424,8657,6678


In [4]:
smart.City.unique()

array(['Oslo', 'Bergen', 'Amsterdam', 'Copenhagen', 'Stockholm',
       'Montreal', 'Vienna', 'Odense', 'Singapore', 'Boston', 'Zurich',
       'Trondheim', 'Vasteras', 'Aalborg', 'Ottawa', 'Washington, DC',
       'Stavanger', 'Los Angeles', 'Helsinki', 'Vancouver', 'Berlin',
       'Toronto', 'New York', 'Chicago', 'Taipei', 'Dusseldorf',
       'Reykjavik', 'Lyon', 'Helsingborg', 'Munich', 'Philadelphia',
       'Hamburg', 'Geneva', 'Luxembourg', 'Aarhus', 'Turku', 'Espoo',
       'Paris', 'Hannover', 'Strasbourg', 'Joensuu', 'Sydney',
       'Frankfurt am Main', 'San Francisco', 'Oulu', 'Cologne', 'Bochum',
       'Stuttgart', 'Hameenlinna', 'London', 'Tampere', 'Adelaide',
       'Melbourne', 'Tokyo', 'Jyvaskyla', 'Perth', 'Vantaa', 'Auckland',
       'Osaka', 'Bordeaux', 'Milan', 'Madrid', 'Daejeon', 'Nantes',
       'Brussels', 'Marseille', 'Leeds', 'Lahti', 'Dublin', 'Abu Dhabi',
       'Verona', 'Seoul', 'Nice', 'Tallinn', 'Dubai', 'Catania',
       'Bayreuth', 'Turin', 'Flore

In [5]:
smart.Country.unique()

array(['Norway', 'Netherlands', 'Denmark', 'Sweden', 'Canada', 'Austria',
       'Singapore', 'United States Of America', 'Switzerland', 'Finland',
       'Germany', 'Taiwan', 'Iceland', 'France', 'Luxembourg',
       'Australia', 'United Kingdom', 'Japan', 'New Zealand', 'Italy',
       'Spain', 'South Korea', 'Belgium', 'Ireland',
       'United Arab Emirates', 'Estonia', 'Hungary', 'Slovakia',
       'Slovenia', 'Israel', 'China', 'Portugal', 'Czech Republic',
       'Malaysia', 'Russia', 'Latvia'], dtype=object)

# 3. Request data from APIS

## 3.1 API COUNTRY - CONTINENTS

In [6]:
import requests

url = "https://referential.p.rapidapi.com/v1/country"

querystring = {"fields":"currency,currency_num_code,currency_code,continent_code,currency,iso_a3,dial_code"}

headers = {
    'x-rapidapi-host': "referential.p.rapidapi.com",
    'x-rapidapi-key': "33e030ac4cmshbfb8d0e76a97e42p1fe169jsn3da51bbbf4d4"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [7]:
df = json_normalize(response.json())

In [8]:
# Drop unnecessary columns
df.drop(columns=[
    'iso_a3','currency','currency_code','key', 'currency_num_code'],inplace=True)

In [9]:
df.sample()

,continent_code,value
148,EU,Romania


In [10]:
df["continent_code"].unique()

array(['AS', 'EU', 'AF', 'NA', 'SA', 'OC'], dtype=object)

In [11]:
df.value.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua And Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia And Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic',
       'Chad', 'Chile', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia',
       'Cuba', 'Curaçao', 'Turkish Republic Of Northern Cyprus',
       'Czech Republic', 'Democratic Republic Of The Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia',
       'Federated States Of Micronesia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenla

In [ ]:
#df_smart = smart.merge(df,left_on="Country", right_on="value", suffixes=["_izq", "_dcha"])


In [12]:
df_smart = df.merge(smart,left_on="value", right_on="Country", suffixes=["_izq", "_dcha"])
#df_smart.sample()

In [13]:
df_smart

,continent_code,value,Unnamed: 0,City,Country,Smart_Mobility,Smart_Environment,Smart_Government,Smart_Economy,Smart_People,Smart_Living,SmartCity_Index
0,OC,Australia,41,Sydney,Australia,6263,3890,5932,5075,7305,9620,6442
1,OC,Australia,51,Adelaide,Australia,6313,4148,5792,3410,4678,9620,5892
2,OC,Australia,52,Melbourne,Australia,7230,3906,5848,2780,5188,9620,6061
3,OC,Australia,55,Perth,Australia,6677,4026,5134,3370,4810,9620,5885
4,EU,Austria,6,Vienna,Austria,5683,7608,6232,5415,8580,7500,6771
...,...,...,...,...,...,...,...,...,...,...,...,...
94,NA,United States Of America,17,Los Angeles,United States Of America,7745,4574,4750,7235,7498,6220,6437
95,NA,United States Of America,22,New York,United States Of America,7607,4468,6038,6520,6610,6220,6336
96,NA,United States Of America,23,Chicago,United States Of America,8110,4680,5170,7265,7123,6220,6551
97,NA,United States Of America,30,Philadelphia,United States Of America,7478,4800,5356,7060,6930,6220,6389


In [14]:
df_smart["continent_code"].unique()

array(['OC', 'EU', 'NA', 'AS'], dtype=object)

In [15]:
df_smart.drop(columns=['value'], inplace = True)

In [16]:
continent_code_rename = {"continent_code": "Continent"}
df_smart.rename(columns=continent_code_rename, inplace=True)

In [17]:
df_smart.shape

(99, 11)

In [18]:
df_smart.isnull().sum()

Continent            0
Unnamed: 0           0
City                 0
Country              0
Smart_Mobility       0
Smart_Environment    0
Smart_Government     0
Smart_Economy        0
Smart_People         0
Smart_Living         0
SmartCity_Index      0
dtype: int64

In [19]:
df_smart.to_csv("../data/continent_smart.csv")

## 3.1 API LIFE QUALITY

In [ ]:
smart.City.unique()

In [ ]:
# as the API only recives certain names in lowercase, I´m going to change the city names
# into lowercase

In [ ]:
#cities_lower = map(lambda city:city.lower(),smart.City)
print(list(cities_lower))

In [ ]:
df_smart["City"] = df_smart["City"].apply(cf.lowercase)

In [ ]:
"""dict_rename = {
    "dÃ\x83Â¼sseldorf" : "dusseldorf",
    "vã\x83â¤sterã\x83â¥s" : "vasteras", V%C3%A4ster%C3%A5s
    "hã\x83â¤meenlinna" : "hameenlinna",
    "jyvã\x83â¤skylã\x83â¤" : "jyväskylä"
}
df_smart.rename(df_smart["City"] == dict_rename, inplace=True)
df_smart.head()
"""

In [ ]:
df_smart.City.unique()

In [ ]:
cities = list(smart.City.unique())

In [ ]:
scores =[]
for city in cities:
    response = requests.get(f"https://api.teleport.org/api/urban_areas/slug:{city}/scores/").json().["teleport_city_score"]
    scores.append(response)

##### EJEMPLO OSLO

In [20]:
oslo = "https://api.teleport.org/api/urban_areas/slug:oslo/scores/"

In [21]:
response_oslo = requests.get(oslo).json()

In [22]:
response_oslo

{'_links': {'curies': [{'href': 'https://developers.teleport.org/api/resources/Location/#!/relations/{rel}/',
    'name': 'location',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/City/#!/relations/{rel}/',
    'name': 'city',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/UrbanArea/#!/relations/{rel}/',
    'name': 'ua',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/Country/#!/relations/{rel}/',
    'name': 'country',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/Admin1Division/#!/relations/{rel}/',
    'name': 'a1',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/Timezone/#!/relations/{rel}/',
    'name': 'tz',
    'templated': True}],
  'self': {'href': 'https://api.teleport.org/api/urban_areas/slug:oslo/scores/'}},
 'categories': [{'color': '#f3c32c',
   'name': 'Housing',
   'score_out_of_10': 3.768},


In [23]:
json_normalize(response_oslo)

,categories,summary,teleport_city_score,_links.curies,_links.self.href
0,"[{'color': '#f3c32c', 'name': 'Housing', 'scor...","<p>Oslo, Norway, is among the top cities with ...",64.314595,[{'href': 'https://developers.teleport.org/api...,https://api.teleport.org/api/urban_areas/slug:...


In [24]:
response_oslo["teleport_city_score"]

64.31459459459458

In [25]:
response_oslo["summary"]

'<p>Oslo, Norway, is among the top cities with a <b>free business environment</b>.\n\n    \n        According to our city rankings, this is a good place to live with high ratings in <b>safety</b>, <b>healthcare</b> and <b>environmental quality</b>.\n    \n\n    \n</p>\n\n\n    <p>Oslo is one of the top ten city matches for 0.9% of Teleport users.</p>\n'

In [26]:
response_oslo["summary"].strip()

'<p>Oslo, Norway, is among the top cities with a <b>free business environment</b>.\n\n    \n        According to our city rankings, this is a good place to live with high ratings in <b>safety</b>, <b>healthcare</b> and <b>environmental quality</b>.\n    \n\n    \n</p>\n\n\n    <p>Oslo is one of the top ten city matches for 0.9% of Teleport users.</p>'